<a href="https://colab.research.google.com/github/christophermalone/DSCI325/blob/main/Module3_Part4_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 3 | Part 4 | SQL : Summerize + Several Other Actions in SQL

The Notebook makes extensive use of many SQL statements.  The following SQL statements are covered in this Notebook.


*   Aggregate() actions
*   Table actions such as CREATE, ALTER, DROP
*   Record actions such as UPDATE, INSERT, DELETE



<table width='100%' ><tr><td bgcolor='green'></td></tr></table>

### Example 3.4.P
For this notebook, we will consider data from a pizza place. The following fields are provided in this data.  This data was collected over 2 years -- one dataset for each year.
 
The following 7 fields will be considered here:

*   LocatonID: Unique ID for each pizza store location
*   DeliveryType: Direct (Store completed the delivery) or SubContract (Delivery was subcontracted out)
*   SameZip: Was the delivery address in the same zipcode as the location?
*   Type: Descriptor for how order was obtained (In-Person, Phone / App, Corporate, OtherLocation)
*   Minutes: Minutes to process order and deliver pizza 
*   Comments:  When Minutes over 1 hour, comments are required.


<table width='100%' ><tr><td bgcolor='green'></td></tr></table>

## Making a Connection

Here, an SQLite3 package will be used to connect to the desired database.

In [7]:
import pandas as pd
import sqlite3

# Getting Table Names and Structure

Doing an initial query to identify the tables within this database.


<p align='center'><img src="https://drive.google.com/uc?export=view&id=1CKUvDXY7LYgY4rIbBtIl5E1lw3xtZTDo"></p>






In [8]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "SELECT name FROM sqlite_master WHERE type='table'"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head()

,name
0,sqlite_sequence
1,PizzaDelivery2019
2,PizzaDelivery2020


Next, getting the structure of the the <strong>PizzaDelivery2019</strong> table.

<p align='center'><img src="https://drive.google.com/uc?export=view&id=1lwH1o94ITwuUPzriPkT76J0u3VKD-WEx"></p>



In [9]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "PRAGMA table_info(PizzaDelivery2019)"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(10)

,cid,name,type,notnull,dflt_value,pk
0,0,TableID,INTEGER,0,None,1
1,1,LocationID,VARCHAR,0,None,0
2,2,DeliveryType,VARCHAR,0,None,0
3,3,SameZip,VARCHAR,0,None,0
4,4,Type,VARCHAR,0,None,0
5,5,Minutes,DOUBLE,0,None,0
6,6,Comments,VARCHAR,0,None,0


Next, getting the structure of the the <strong>PizzaDelivery2020</strong> table.

<p align='center'><img src="https://drive.google.com/uc?export=view&id=1iJazgrQapOvRqZi7asWhWyDLlqe-K2BY"></p>



In [10]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "PRAGMA table_info(PizzaDelivery2020)"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(10)

,cid,name,type,notnull,dflt_value,pk
0,0,TableID,INTEGER,0,None,1
1,1,LocationID,VARCHAR,0,None,0
2,2,DeliveryType,VARCHAR,0,None,0
3,3,SameZip,VARCHAR,0,None,0
4,4,Type,VARCHAR,0,None,0
5,5,Quantity,INTEGER,0,None,0
6,6,Minutes,DOUBLE,0,None,0
7,7,Comments,VARCHAR,0,None,0


<strong>Note</strong>: The PizzaDelivery2019 table does <strong>not</strong> contain a Quantity field; whereas, PizzaDelivery2020 table does.  Akin to the R/Python version of this notebook, initial summaries will be done using Minutes.  Recall, these summaries ignore the chained deliveries.  The chained deliveries have the number of deliveries within the Comments field.  Therefore, these summaries will be updated using the Number from the Comments field.

# Initial Summaries using Minutes Field

To begin, suppose the goal is to obtain the average delivery time across all deliveries.

In [17]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT AVG(Minutes) AS [Avg Delivery Time] 
                          FROM PizzaDelivery2019
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head()

,Avg Delivery Time
0,9.624701


Next, get the average delivery time for each delivery type.

In [18]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT AVG(Minutes) AS [Avg Delivery Time]
                          FROM PizzaDelivery2019
                          GROUP BY DeliveryType
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head()

,Avg Delivery Time
0,9.765571
1,8.130779


<strong>Comments</strong>:

1.  Notice that the code above does not properly identify which row is for Direct and which row is for SubContract.  This happened because DeliveryType was not selected in the initial SELECT statment.

2. The ROUND() function can be used to reduce the number of decimal places used in the output.

In [19]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT DeliveryType, ROUND(AVG(Minutes),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2019
                          GROUP BY DeliveryType
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head()

,DeliveryType,Avg Delivery Time
0,Direct,9.8
1,SubContract,8.1


The following will obtain the average delivery time across Delivery Type *and* Samezip.  

In [20]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Samezip, DeliveryType, ROUND(AVG(Minutes),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2019
                          GROUP BY Samezip, DeliveryType
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head()

,SameZip,DeliveryType,Avg Delivery Time
0,No,Direct,9.4
1,No,SubContract,9.5
2,Yes,Direct,10.0
3,Yes,SubContract,6.3


<strong>Note</strong>:  The column order in the output is determined by the SELECT statment -- not the order in which they appear in the GROUP BY statement. (This is different than R and Python).

The following extends the summaries across Type, Samezip, and Delivery Type.

In [21]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type, Samezip, DeliveryType, ROUND(AVG(Minutes),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2019
                          GROUP BY Type, Samezip, DeliveryType
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(16)

,Type,SameZip,DeliveryType,Avg Delivery Time
0,Corporate,No,Direct,11.6
1,Corporate,No,SubContract,11.9
2,Corporate,Yes,Direct,13.0
3,Corporate,Yes,SubContract,6.8
4,In-Person,No,Direct,5.4
5,In-Person,No,SubContract,5.3
6,In-Person,Yes,Direct,5.9
7,In-Person,Yes,SubContract,4.1
8,OtherLocation,No,Direct,17.6
9,OtherLocation,No,SubContract,16.5


The <strong>ORDER BY</strong> statement can be used to sort the rows in the output.  The following use of the ORDER BY statement will allow the output to match R/Python.

In [22]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type, Samezip, DeliveryType, ROUND(AVG(Minutes),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2019
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(16)

,Type,SameZip,DeliveryType,Avg Delivery Time
0,Corporate,No,Direct,11.6
1,In-Person,No,Direct,5.4
2,OtherLocation,No,Direct,17.6
3,Phone / App,No,Direct,9.3
4,Corporate,Yes,Direct,13.0
5,In-Person,Yes,Direct,5.9
6,OtherLocation,Yes,Direct,16.4
7,Phone / App,Yes,Direct,9.5
8,Corporate,No,SubContract,11.9
9,In-Person,No,SubContract,5.3


## Breaking apart a Column

There is a <strong>PIVOT</strong> operation for SQL that is akin to the <strong>spread</strong> operation in Python / R.  However, PIVOT is *not* available in SQLite.

The desired table, i.e. one with Average Delivery Time for Direct and Average Delivery Time for SubContract in seperate columns can be done using a <strong>JOIN</strong>. The JOIN operation will be covered in more detail in a future notebook. 


<p align='center'><img src="https://drive.google.com/uc?export=view&id=1WpdNlDCR5TjerDgV6-TvL_axgTRDfF57"></p>



### Preparing Tables for JOIN

The following steps are necessary to create the desired table using SQL.

1. Obtain the summaries for DeliveryType = Direct
2. Create a new column that contains a unique identifier for each row in the summary table.  This field will be called Key and will be used in the JOIN operation.
3. Obtain the summaries for DeliveryType = SubContract.  Again, create a new column that uniquely identifies each row.
4. JOIN the two tables
5. Create the Difference field and ORDER BY Difference


Step #1: Get appropriate table for DeliveryType = Direct.

In [23]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type, Samezip, DeliveryType, ROUND(AVG(Minutes),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2019
                          WHERE DeliveryType = 'Direct'
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(16)

,Type,SameZip,DeliveryType,Avg Delivery Time
0,Corporate,No,Direct,11.6
1,In-Person,No,Direct,5.4
2,OtherLocation,No,Direct,17.6
3,Phone / App,No,Direct,9.3
4,Corporate,Yes,Direct,13.0
5,In-Person,Yes,Direct,5.9
6,OtherLocation,Yes,Direct,16.4
7,Phone / App,Yes,Direct,9.5


Step #2a: Create a new field called Key that uniquely defines each row.

In [28]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type||SameZip AS [Key], Type, Samezip, DeliveryType, ROUND(AVG(Minutes),1) AS [Direct_Avg]
                          FROM PizzaDelivery2019
                          WHERE DeliveryType = 'Direct'
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(16)

,Key,Type,SameZip,DeliveryType,Direct_Avg
0,CorporateNo,Corporate,No,Direct,11.6
1,In-PersonNo,In-Person,No,Direct,5.4
2,OtherLocationNo,OtherLocation,No,Direct,17.6
3,Phone / AppNo,Phone / App,No,Direct,9.3
4,CorporateYes,Corporate,Yes,Direct,13.0
5,In-PersonYes,In-Person,Yes,Direct,5.9
6,OtherLocationYes,OtherLocation,Yes,Direct,16.4
7,Phone / AppYes,Phone / App,Yes,Direct,9.5


Step #2b: Put this Python dataframe into the PizzaDelivery database.
<ul>
<ul>
  A Python dataframe can be added as a table into our database using the <strong>to_sql</strong> function.

  <p align='center'><font size="+1">df.to_sql(<i>'table name'</i>, <i>db_connection</i>, if_exists='replace')</font></p>

  Line #14 in the following code save the Python dataframe into a table called PDFix in the PizzaDelivery.db.

</ul></ul>

In [29]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type||SameZip AS [Key], Type, Samezip, DeliveryType, ROUND(AVG(Minutes),1) AS [Direct_Avg]
                          FROM PizzaDelivery2019
                          WHERE DeliveryType = 'Direct'
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)

df.to_sql('PD2019_Direct_Summaries',connect_db , if_exists='replace' )

#Closing the connection
connect_db.close()


<ul><ul>
Verify that the database now includes the Pizza Delivery Summary Table for Direct Deliveries.
</ul></ul>


<p align='center'><img src="https://drive.google.com/uc?export=view&id=1hASl-WLmowuS2yW4f7ssBEHb2GvO2C0D"></p>


<table width='100%'><tr><td bgcolor='orange'><font color="white" size="+2">Aside: Creating Table via Native SQL</font></td></tr></table>

In the code above, the df.to_sql() function is writing the output of the resulting SQL code into a table called PD2019_Direct_Summaries.  When working in native SQL, this can be accomplished using the following.

<p align='center'><font size="+3">CREATE TABLE <i> tablename </i> AS </font></p>



<p align='center'><img src="https://drive.google.com/uc?export=view&id=1Et31xp0JeHg-TpFlMg_hucyICCzQ3GH9" width="50%" height="50%"></p>

<table width='100%'><tr><td bgcolor='orange'>&nbsp;</td></tr></table>

In [30]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "SELECT name FROM sqlite_master WHERE type='table'"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head()

,name
0,sqlite_sequence
1,PizzaDelivery2019
2,PizzaDelivery2020
3,PD2019_Direct_Summaries


<ul><ul>
Also, verify the struture of this newly created table in our database.
</ul></ul>

In [31]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "PRAGMA table_info(PD2019_Direct_Summaries)"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(10)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Key,TEXT,0,None,0
2,2,Type,TEXT,0,None,0
3,3,SameZip,TEXT,0,None,0
4,4,DeliveryType,TEXT,0,None,0
5,5,Direct_Avg,REAL,0,None,0


Step #3: Do all the same for DeliveryType = SubContract; Including creating a unique identifer for each row, and saving this output into a data table within our database.

In [32]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type||SameZip AS [Key], Type, Samezip, DeliveryType, ROUND(AVG(Minutes),1) AS [SubContract_Avg]
                          FROM PizzaDelivery2019
                          WHERE DeliveryType = 'SubContract'
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)

df.to_sql('PD2019_SubContract_Summaries',connect_db , if_exists='replace' )

#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(20)

,Key,Type,SameZip,DeliveryType,SubContract_Avg
0,CorporateNo,Corporate,No,SubContract,11.9
1,In-PersonNo,In-Person,No,SubContract,5.3
2,OtherLocationNo,OtherLocation,No,SubContract,16.5
3,Phone / AppNo,Phone / App,No,SubContract,10.1
4,CorporateYes,Corporate,Yes,SubContract,6.8
5,In-PersonYes,In-Person,Yes,SubContract,4.1
6,OtherLocationYes,OtherLocation,Yes,SubContract,11.9
7,Phone / AppYes,Phone / App,Yes,SubContract,6.6


<ul><ul>
Verify that the summary table for Delivery = SubContract was successfully created in our database.
</ul></ul>

In [33]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "SELECT name FROM sqlite_master WHERE type='table'"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head()

,name
0,sqlite_sequence
1,PizzaDelivery2019
2,PizzaDelivery2020
3,PD2019_Direct_Summaries
4,PD2019_SubContract_Summaries


<ul><ul>
Doing a quick check of the struture of the summary table for Delivery = SubContract.
</ul></ul>

In [35]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "PRAGMA table_info(PD2019_SubContract_Summaries)"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(10)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Key,TEXT,0,None,0
2,2,Type,TEXT,0,None,0
3,3,SameZip,TEXT,0,None,0
4,4,DeliveryType,TEXT,0,None,0
5,5,SubContract_Avg,REAL,0,None,0


### Process for JOIN 

<ol>
 <li>Identify the two tables to be joined together.  Often the two tables are identified as LEFT and RIGHT.  In addition, identify which fields are needed to complete the desired JOIN.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=1hCQTNn7D1OIi8R6DQQk7NTXRuzZPOIhY" width="75%" height="75%"></p>
</li>

<li>The <strong>Key</strong> field will be used to identify the rows used for the matching across the two tables.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=1vFR3eLQgAcbemZHiuqKHTwjW9zBACQ-G" width="75%" height="75%"></p>
</li>

<li>The next step is to PUT the desired columns from the RIGHT table into the LEFT table.  This step should be done carefully as variable names in the final table may not longer be unique.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=17Jx2M4oSKSza4AWZMdbAK7_TEGA7s7j0" width="75%" height="75%"></p>
</li>

<li>Repeat this process for all rows in the LEFT table.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=18-seNKTDjkR5u7QwXGFKvJ70_6t_y4ZQ" width="75%" height="75%"></p>
</li>

<li>The JOIN results in the following table.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=14L4LVga_lAZq5VpMaPXNMSmqe-LwxMCq" width="50%" height="50%"></p>
</li>

<li>The last step is to remove the Key field as this was only used to JOIN the two tables together.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=1PuV1TVvv59aFGAWDqifGAC2SQguaFwDt" width="50%" height="50%"></p>
</li>


### The SQL Code for JOIN

The <strong>JOIN</strong> process described above relates to the SQL code below in the following manner.


*   The LEFT table is defined to be the PD2019_Direct_Summaries table (which is LEFT or RIGHT does matter in this case)
*   The RIGHT table is defined to be the PD2019_SubContract_Summaries table
*   The JOIN will be done using the Key field



<p align="center">
<font size="+2"><i>Table A</i> LEFT JOIN <i>Table B</i> USING(<i>key</i>)</font>
</p>

In [36]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT *
                          FROM PD2019_Direct_Summaries
                          LEFT JOIN (
                                      SELECT Key, SubContract_Avg
                                      FROM PD2019_SubContract_Summaries
                                    )
                          USING(Key)
                      """
                          , connect_db)

#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(20)

,index,Key,Type,SameZip,DeliveryType,Direct_Avg,SubContract_Avg
0,0,CorporateNo,Corporate,No,Direct,11.6,11.9
1,1,In-PersonNo,In-Person,No,Direct,5.4,5.3
2,2,OtherLocationNo,OtherLocation,No,Direct,17.6,16.5
3,3,Phone / AppNo,Phone / App,No,Direct,9.3,10.1
4,4,CorporateYes,Corporate,Yes,Direct,13.0,6.8
5,5,In-PersonYes,In-Person,Yes,Direct,5.9,4.1
6,6,OtherLocationYes,OtherLocation,Yes,Direct,16.4,11.9
7,7,Phone / AppYes,Phone / App,Yes,Direct,9.5,6.6


Next, adding a second query to prettify output as desired.

In [37]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT 
                            Type, 
                            SameZip, 
                            Direct_Avg AS [Direct], 
                            SubContract_Avg AS [SubContract],
                            Direct_Avg - SubContract_Avg AS [Difference]
                          FROM
                             ( 
                               SELECT *
                               FROM PD2019_Direct_Summaries
                               LEFT JOIN (
                                          SELECT Key, SubContract_Avg
                                          FROM PD2019_SubContract_Summaries
                                        )
                               USING(Key)
                            )
                          ORDER BY Difference
                      """
                          , connect_db)


#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(20)

,Type,SameZip,Direct,SubContract,Difference
0,Phone / App,No,9.3,10.1,-0.8
1,Corporate,No,11.6,11.9,-0.3
2,In-Person,No,5.4,5.3,0.1
3,OtherLocation,No,17.6,16.5,1.1
4,In-Person,Yes,5.9,4.1,1.8
5,Phone / App,Yes,9.5,6.6,2.9
6,OtherLocation,Yes,16.4,11.9,4.5
7,Corporate,Yes,13.0,6.8,6.2


Note: Aside from some minor differences in rounding, the table above matches the table obtained from Python.


<p align='center'><img src="https://drive.google.com/uc?export=view&id=1WpdNlDCR5TjerDgV6-TvL_axgTRDfF57"></p>

## Fixing the Basic Summaries - Getting Number from Comments Field

Recall, for deliveries that took longer than 60 minutes a comment was required. Some of these deliveries where chained deliveries.  In this situation, the number of deliveries chained together is given by Number.

In [38]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Comments 
                          FROM PizzaDelivery2019
                          WHERE INSTR(Comments, "=") > 0
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(200)

,Comments
0,Chained Delivery; Number = 3; Changed Order;
1,Chained Delivery; Number = 6;
2,Chained Delivery; Number = 6;
3,Chained Delivery; Number = 4; Changed Order;
4,Chained Delivery; Number = 4;
5,Chained Delivery; Number = 3; Changed Order; W...
6,Chained Delivery; Number = 3; Changed Order;
7,Chained Delivery; Number = 5; Wrong Address;
8,Chained Delivery; Number = 2; Changed Order; W...
9,Chained Delivery; Number = 4; Wrong Address;


<font size="+2"><strong>Issue</strong></font>

A few of deliveries from 2019 are chained deliveries in which more than one delivery was made at once. Locations are not supposed to do this as this is against company policy.   The Number in the Comment field indicates how many deliveries were made in this chained delivery.

There are two approaches that will be discussed here to fix this issue.

1.  A superficial fix that only effects the output
2.  A permanent fix that adds a Quantity column to the table.  (This approach is desired as the table will then match the struture of the 2020 table.)

## Superficial Fix -- Only Effects Output

The <strong>INSTR()</strong> function can be used to pull out the Number from the Comments field.

In [39]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT TableID, Comments,
                          CASE
                             WHEN INSTR(Comments,"=") > 0 THEN SUBSTR(Comments, INSTR(Comments,"=")+2,1)
                             ELSE 1
                          END AS [Number]
                          FROM PizzaDelivery2019
                          WHERE INSTR(Comments,"=") > 0
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(200)

,TableID,Comments,Number
0,6821,Chained Delivery; Number = 3; Changed Order;,3
1,6826,Chained Delivery; Number = 6;,6
2,6828,Chained Delivery; Number = 6;,6
3,6832,Chained Delivery; Number = 4; Changed Order;,4
4,6855,Chained Delivery; Number = 4;,4
5,7746,Chained Delivery; Number = 3; Changed Order; W...,3
6,7747,Chained Delivery; Number = 3; Changed Order;,3
7,7748,Chained Delivery; Number = 5; Wrong Address;,5
8,7749,Chained Delivery; Number = 2; Changed Order; W...,2
9,7752,Chained Delivery; Number = 4; Wrong Address;,4


Now that the SQL code correctly identifies the Number of deliveries for each row in the table, a second query will be added to complete the desired summaries for each Type - SameZip - DeliveryType combinations.

In [40]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type, SameZip, DeliveryType, ROUND(SUM(Minutes)/SUM(Number),1) AS [Avg Delivery Time]
                          FROM
                          (
                            SELECT Type, SameZip, DeliveryType, Minutes, Comments,
                                   CASE
                                      WHEN INSTR(Comments,"=") > 0 THEN SUBSTR(Comments, INSTR(Comments,"=")+2,1)
                                    ELSE 1
                                  END AS [Number]
                            FROM PizzaDelivery2019
                          )
                          GROUP BY Type, SameZip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(200)

,Type,SameZip,DeliveryType,Avg Delivery Time
0,Corporate,No,Direct,11.6
1,In-Person,No,Direct,5.4
2,OtherLocation,No,Direct,17.6
3,Phone / App,No,Direct,9.3
4,Corporate,Yes,Direct,12.9
5,In-Person,Yes,Direct,5.9
6,OtherLocation,Yes,Direct,16.1
7,Phone / App,Yes,Direct,9.4
8,Corporate,No,SubContract,11.9
9,In-Person,No,SubContract,5.3


## Permanent Fix -- Add Quantity field to table in database

The <strong>ADD COLUMN</strong> operation can be used to add a column to an exsisting table.  When adding a field to existing table, it is important to specify the desired structure for the new field. For example, the Quantity column should be specifed as interger as this number of deliveries will be of an interger data type.  Also, for most deliveries the number of deliveries is 1; thus, the default value will be specifed as 1. 

<p align="center"><font size="+2">
ADD COLUMN Quantity INT DEFAULT 1
</font></p>

This ADD COLUMN operation is part of <strong>ALTER TABLE</strong>.  Consider the following syntax to add this column to the PizzaDelivery2019 data table.


Note: You may get a NoneType error here; this error is can be ignored.


In [41]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                                ALTER TABLE PizzaDelivery2019
                                ADD COLUMN Quantity INT DEFAULT 1;
                             """
                            )
                       
#Closing the connection
dbconnection.close()


Taking a look at newly created column when Comments contain an "=" sign and for a few additional records, i.e. records with TableIDs ranging between 6821 - 6826.

In [42]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT TableID, Quantity, Comments 
                          FROM PizzaDelivery2019
                          WHERE INSTR(Comments,"=") > 0 OR TableID IN (6821, 6822,6823,6824,6825, 6826)
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(200)

,TableID,Quantity,Comments
0,6821,1,Chained Delivery; Number = 3; Changed Order;
1,6822,1,
2,6823,1,
3,6824,1,Customer Delayed; Changed Order;
4,6825,1,
5,6826,1,Chained Delivery; Number = 6;
6,6828,1,Chained Delivery; Number = 6;
7,6832,1,Chained Delivery; Number = 4; Changed Order;
8,6855,1,Chained Delivery; Number = 4;
9,7746,1,Chained Delivery; Number = 3; Changed Order; W...


Again, using the INSTR() function to pull of the appropriate Number from the Comments field.  The <strong>to_sql()</strong> is used to put this dataframe into the PizzaDelivery database as a new table.


In [43]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT TableID,
                                 CASE
                                    WHEN INSTR(Comments,"=") > 0 THEN SUBSTR(Comments, INSTR(Comments,"=")+2,1)
                                    ELSE 1
                                END AS [Number]
                          FROM PizzaDelivery2019
                          WHERE INSTR(Comments,"=") > 0
                       """
                          , connect_db)

df.to_sql('PD2019_QuantityFix',connect_db , if_exists='replace' )


#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(200)

,TableID,Number
0,6821,3
1,6826,6
2,6828,6
3,6832,4
4,6855,4
5,7746,3
6,7747,3
7,7748,5
8,7749,2
9,7752,4


A new table should have been added to the database.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=1GpdGFRY4scoziRPDZLfy6dtmyw3OBPY7" width="65%" height="65%"></p>


In [44]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "SELECT name FROM sqlite_master WHERE type='table'"
                          , dbconnection)
#Closing the connection
dbconnection.close()

#Using pandas to show output
df.head(10)

,name
0,sqlite_sequence
1,PizzaDelivery2019
2,PizzaDelivery2020
3,PD2019_Direct_Summaries
4,PD2019_SubContract_Summaries
5,PD2019_QuantityFix


The final step is to replace the Quantity values in the PizzaDelivery2019 table with the Number values from the PD2019_QuantityFix table when appropriate.  A description of this process is provided here.

<ol>
<li> Consider two tables.  The TableID from the PizzaDelivery table will need to be compared against the list of TableID's in the PD2019_QuantityFix table.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=1l3E4pXk4pbXKIhuCGj1BlXCCsqLOolgB" width="50%" height="50%"></p>
</li>

<li>If the TableID from the PizzaDelivery table does not exists in the PD2019_QuantityFix table, then leave the Quantity value as 1.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=1PvaW7cfRWL7pICtUpcLfmepv29j1Vn99" width="50%" height="50%"></p>
</li>

<li>In the situation where the TableID from the PizzaDelivery table exists in the PD2019_QuantityFix table, then <strong>UPDATE</strong> the Quantity value with Number.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=18yi3HNMJBYmXNPc_mj85KzXngURePZDW" width="50%" height="50%"></p>
</li>
</ol>


### UPDATE Command in SQL

The <strong>UPDATE</strong> statement can be used in SQL to update a record, i.e. row.  The UPDATE is only to be done on TableIDs that exist in the PD2019_QuantityFix table.  UPDATE statement has the following structure.

<p align='center'>
<font size ="+2">
UPDATE <i>Tablename</i> SET <i>Quantity</i> = _____ WHERE ______
</font>
</p>

<strong>Comments</strong>:
1.  The *updated* Quantity value will be equal to Number from the PD2019_QuantityFix table
2.  The WHERE clause should only include TableIDs that exist in the PD2019_QuantityFix table
3.  The <strong>.commit()</strong> function *must* be run in order for the UPDATE to take effect (See Line #17).  This function must also be used when using the INSERT or DELETE statements as well.

In [45]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                               UPDATE PizzaDelivery2019 SET Quantity = (
                                                                        SELECT Number
                                                                        FROM PD2019_QuantityFix
                                                                        WHERE PizzaDelivery2019.TableID = PD2019_QuantityFix.TableID
                                                                       )
                                WHERE PizzaDelivery2019.TableID IN (
                                                                    SELECT TableID
                                                                    FROM PD2019_QuantityFix
                                                                   )
                               """
                             )
dbconnection.commit()
                       
#Closing the connection
dbconnection.close()

Doing a check to see if appropriate records were updated properly.

In [46]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT TableID, Quantity, Comments 
                          FROM PizzaDelivery2019
                          WHERE INSTR(Comments,"=") > 0 OR TableID IN (6821, 6822,6823,6824,6825, 6826)
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(20)

,TableID,Quantity,Comments
0,6821,3,Chained Delivery; Number = 3; Changed Order;
1,6822,1,
2,6823,1,
3,6824,1,Customer Delayed; Changed Order;
4,6825,1,
5,6826,6,Chained Delivery; Number = 6;
6,6828,6,Chained Delivery; Number = 6;
7,6832,4,Chained Delivery; Number = 4; Changed Order;
8,6855,4,Chained Delivery; Number = 4;
9,7746,3,Chained Delivery; Number = 3; Changed Order; W...


### Getting Average Minutes using Quantity field for 2019

In [47]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type, Samezip, DeliveryType, ROUND(SUM(Minutes)/SUM(Quantity),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2019
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(16)

,Type,SameZip,DeliveryType,Avg Delivery Time
0,Corporate,No,Direct,11.6
1,In-Person,No,Direct,5.4
2,OtherLocation,No,Direct,17.6
3,Phone / App,No,Direct,9.3
4,Corporate,Yes,Direct,12.9
5,In-Person,Yes,Direct,5.9
6,OtherLocation,Yes,Direct,16.1
7,Phone / App,Yes,Direct,9.4
8,Corporate,No,SubContract,11.9
9,In-Person,No,SubContract,5.3


Now that the Quantity field has been properly added to the PizzaDelivery2019 table, the PD2019_QuanityFix table can be removed from the database.  The following <strong>DROP TABLE</strong> statement can be used to drop a table.

In [48]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                               DROP TABLE PD2019_QuantityFix
                              """
                      )
                       
#Closing the connection
dbconnection.close()

A quick check to verify that the table was indeed removed from the database.

In [49]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "SELECT name FROM sqlite_master WHERE type='table'"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(10)

,name
0,sqlite_sequence
1,PizzaDelivery2019
2,PizzaDelivery2020
3,PD2019_Direct_Summaries
4,PD2019_SubContract_Summaries


## Some Utility Functions in SQL

<table width='100%'><tr><td bgcolor='orange'>&nbsp;</td></tr></table>

The following <strong>CREATE TABLE</strong> statement can be used to create a new table from an existing table.

In [ ]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                                CREATE TABLE PizzaDelivery2019_v2 AS 
                                SELECT TableID, LocationID, DeliveryType, SameZip, Type, Minutes, Comments
                                FROM PizzaDelivery2019
                              """
                             )
                       
#Closing the connection
dbconnection.close()

The following <strong>DROP TABLE</strong> statement can be used to delete a table from the database. 

Note:  Use DROP TABLE with caution as the table will be permanently deleted from the database.

In [ ]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                               DROP TABLE PizzaDelivery2019
                              """
                      )
                       
#Closing the connection
dbconnection.close()

The following <strong>ALTER TABLE</strong> statement can be used to *rename* an existing table in the database.

In [ ]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                                 ALTER TABLE PizzaDelivery2019_v2 RENAME TO PizzaDelivery2019
                              """
                      )
                       
#Closing the connection
dbconnection.close()

The following <strong>DELETE</strong> statement can be used to delete rows in a database table.

In [ ]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                                 DELETE FROM PizzaDelivery2019_v2 WHERE Type = 'In-Person'
                              """
                      )
dbconnection.commit()
                       
#Closing the connection
dbconnection.close()

<table width='100%'><tr><td bgcolor='orange'>&nbsp;</td></tr></table>

## Analysis for 2020

The analysis in SQL for 2020 is *much* easier as the Quantity column already exists in the database table.  The online data collection form was updated -- Quantity was a required field and the default value for Quantity was 1.

In [51]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT Type, Samezip, DeliveryType, ROUND(SUM(Minutes)/SUM(Quantity),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2020
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(16)

,Type,SameZip,DeliveryType,Avg Delivery Time
0,Corporate,No,Direct,17.1
1,In-Person,No,Direct,8.0
2,OtherLocation,No,Direct,25.0
3,Phone / App,No,Direct,15.1
4,Corporate,Yes,Direct,18.3
5,In-Person,Yes,Direct,9.4
6,OtherLocation,Yes,Direct,24.5
7,Phone / App,Yes,Direct,16.8
8,Corporate,No,Subcontract,7.0
9,In-Person,No,Subcontract,3.0


## Append the Two Summary Tables Together

The last section in the Notebook will consider how to append the two summaries tables together.  This can be done in two ways.

1. The APPEND can be done in Python using the bind_rows() function.
2. THe APPEND can be done by creating a table that contains the summaries from both years.

### Doing the APPEND in Python via bind_rows()

First, recreate the summaries for 2019. Here a new column called Year will be added to differntiate between the two sets of summaries.

In [52]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df_2019 = pd.read_sql_query("""
                          SELECT '2019' AS [Year], Type, Samezip, DeliveryType, ROUND(SUM(Minutes)/SUM(Quantity),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2019
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df_2019.head(16)

,Year,Type,SameZip,DeliveryType,Avg Delivery Time
0,2019,Corporate,No,Direct,11.6
1,2019,In-Person,No,Direct,5.4
2,2019,OtherLocation,No,Direct,17.6
3,2019,Phone / App,No,Direct,9.3
4,2019,Corporate,Yes,Direct,12.9
5,2019,In-Person,Yes,Direct,5.9
6,2019,OtherLocation,Yes,Direct,16.1
7,2019,Phone / App,Yes,Direct,9.4
8,2019,Corporate,No,SubContract,11.9
9,2019,In-Person,No,SubContract,5.3


Next, obtain the summaries for 2020.  Place the outcomes for 2020 into a different Panda dataframe and again create a column for Year akin to what was done with 2019.

In [53]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df_2020 = pd.read_sql_query("""
                          SELECT '2020' AS [Year], Type, Samezip, DeliveryType, ROUND(SUM(Minutes)/SUM(Quantity),1) AS [Avg Delivery Time]
                          FROM PizzaDelivery2020
                          GROUP BY Type, Samezip, DeliveryType
                          ORDER BY DeliveryType, Samezip, Type
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df_2020.head(16)

,Year,Type,SameZip,DeliveryType,Avg Delivery Time
0,2020,Corporate,No,Direct,17.1
1,2020,In-Person,No,Direct,8.0
2,2020,OtherLocation,No,Direct,25.0
3,2020,Phone / App,No,Direct,15.1
4,2020,Corporate,Yes,Direct,18.3
5,2020,In-Person,Yes,Direct,9.4
6,2020,OtherLocation,Yes,Direct,24.5
7,2020,Phone / App,Yes,Direct,16.8
8,2020,Corporate,No,Subcontract,7.0
9,2020,In-Person,No,Subcontract,3.0


Use <strong>dfply</strong> to APPEND the rows together with the bind_rows() function.

In [54]:
pip install dfply


     |████████████████████████████████| 612 kB 5.4 MB/s 


Next, import the dfply package.

In [55]:
from dfply import *

Finally, complete the APPEND using bind_rows() and prettify the output.

In [56]:
Outcomes = (
             df_2019
              >> bind_rows(df_2020)
           )

Outcomes = Outcomes
print(Outcomes.to_string(index=False))

Year          Type SameZip DeliveryType  Avg Delivery Time
2019     Corporate      No       Direct               11.6
2019     In-Person      No       Direct                5.4
2019 OtherLocation      No       Direct               17.6
2019   Phone / App      No       Direct                9.3
2019     Corporate     Yes       Direct               12.9
2019     In-Person     Yes       Direct                5.9
2019 OtherLocation     Yes       Direct               16.1
2019   Phone / App     Yes       Direct                9.4
2019     Corporate      No  SubContract               11.9
2019     In-Person      No  SubContract                5.3
2019 OtherLocation      No  SubContract               16.5
2019   Phone / App      No  SubContract               10.1
2019     Corporate     Yes  SubContract                6.8
2019     In-Person     Yes  SubContract                4.1
2019 OtherLocation     Yes  SubContract               11.9
2019   Phone / App     Yes  SubContract                6

### Doing the APPEND in SQL

An <strong>APPEND</strong> in SQL will be done via the follwoing process.

1.  First, create a table called Summaries to contains the summaries from 2109.
2.  Next, create a table of summaries from 2020 and INSERT these rows into the Summaries table.

Step #1:  Obtain the summaries from 2019 and place these outcomes into a new table called Summaries.

In [ ]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                                CREATE TABLE Summaries AS   
                                             SELECT '2019' AS [Year], Type, Samezip, DeliveryType, ROUND(SUM(Minutes)/SUM(Quantity),1) AS [AvgDeliveryTime]
                                             FROM PizzaDelivery2019
                                             GROUP BY Type, Samezip, DeliveryType
                                             ORDER BY DeliveryType, Samezip, Type
                              """
                             )
                       
#Closing the connection
dbconnection.close()

A quick check to see if table was successfully added to the database.

In [61]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query(
                        "SELECT name FROM sqlite_master WHERE type='table'"
                          , connect_db)
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(10)

,name
0,sqlite_sequence
1,PizzaDelivery2019
2,PizzaDelivery2020
3,PD2019_Direct_Summaries
4,PD2019_SubContract_Summaries
5,Summaries


A simple SELECT * statement can be used to retrieve the contents of the Summaries table.



In [62]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT * 
                          FROM Summaries                          
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(20)

,Year,Type,SameZip,DeliveryType,AvgDeliveryTime
0,2019,Corporate,No,Direct,11.6
1,2019,In-Person,No,Direct,5.4
2,2019,OtherLocation,No,Direct,17.6
3,2019,Phone / App,No,Direct,9.3
4,2019,Corporate,Yes,Direct,12.9
5,2019,In-Person,Yes,Direct,5.9
6,2019,OtherLocation,Yes,Direct,16.1
7,2019,Phone / App,Yes,Direct,9.4
8,2019,Corporate,No,SubContract,11.9
9,2019,In-Person,No,SubContract,5.3


### INSERT Statement in SQL

Step #2: Next, we will create the summaries from 2020 and <strong>INSERT</strong> these outcomes in the Summaries table.  The field names in each of the two tables shoudl match to ensure the INSERT is done correctly.
<p align='center'><img src="https://drive.google.com/uc?export=view&id=1oLsXMnqn6bZvkWGowdJIBkpuBBKjC9eO" width="50%" height="50%"></p>


The <strong>INSERT</strong> statment can be used to insert records into an exsiting table.  The INSERT statement can be used to add a *single* record into an exsiting table or multiple records.  The INSERT statement used here will insert the summaries from 2020 into the Summaries table.



In [64]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                                INSERT INTO Summaries
                                            SELECT '2020' AS [Year], Type, Samezip, DeliveryType, ROUND(SUM(Minutes)/SUM(Quantity),1) AS [AvgDeliveryTime]
                                            FROM PizzaDelivery2020
                                            GROUP BY Type, Samezip, DeliveryType
                                            ORDER BY DeliveryType, Samezip, Type
                       """
                        )
dbconnection.commit()

#Closing the connection
dbconnection.close()


<strong>Note</strong>:  If the above code is run a second time, the INSERT will happen again and the 2020 outcomes will appear <strong>twice</strong> in the Summaries table.  To alleviate this problem, the <strong>DELETE</strong> statement can be used to remove records from the database.

In [ ]:
#Making a connection
dbconnection = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
dbconnection.cursor().execute("""
                                 DELETE FROM Summaries WHERE Year = '2020'
                              """
                      )

dbconnection.commit()
                       
#Closing the connection
dbconnection.close()

Again, a simple SELECT * statement can be used to retrieve the contents of the Summaries table to ensure the 2020 summaries were successfully inserted into this table.

In [ ]:
#Making a connection
connect_db = sqlite3.connect("/content/sample_data/PizzaDelivery.db")

#SQL Statement
df = pd.read_sql_query("""
                          SELECT * 
                          FROM Summaries                          
                       """
                          , connect_db)
                       
#Closing the connection
connect_db.close()

#Using pandas to show output
df.head(50)

,Year,Type,SameZip,DeliveryType,AvgDeliveryTime
0,2019,Corporate,No,Direct,11.6
1,2019,In-Person,No,Direct,5.4
2,2019,OtherLocation,No,Direct,17.6
3,2019,Phone / App,No,Direct,9.3
4,2019,Corporate,Yes,Direct,12.9
5,2019,In-Person,Yes,Direct,5.9
6,2019,OtherLocation,Yes,Direct,16.1
7,2019,Phone / App,Yes,Direct,9.4
8,2019,Corporate,No,SubContract,11.9
9,2019,In-Person,No,SubContract,5.3


<strong>Some Final Comments</strong>:  
1.  The Summaries table does **not** contain a primary key -- which is not good.  
2. A primary key is important and is used to uniquely identify a record.  A primary key can simply be an interger that is automatically incremented by 1 when a new record is added.  
3. A total of four fields must be concatenated together in order to uniquely define a record the Summaries table.  That is, a Year||Type||SameZip||DeliveryType combination is needed to uniquely identify a record in this table -- such an arrangement is not preferred. 
4.  Consider once again the issue of obtaining repeated outcomes from 2020 when the INSERT code chunk is run a second time.  A well-constructed primary key would likely prevent duplicates in a table. 